In [65]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, GlobalMaxPooling1D
from sklearn.metrics import classification_report

In [66]:
def read_tbl(file_path):
    df = pd.read_csv(file_path)
    return df

def process_csv_files(directory_path):
    domestic_df = pd.DataFrame()
    non_domestic_df = pd.DataFrame()

    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            df = read_tbl(file_path)

            df = df[~((df.gist.fillna("") == ""))]
            
            domestic_target_df = df[df.gist.fillna("").str.contains("domestic", case=False)]
            domestic_target_df = domestic_target_df[["gist"]]
            domestic_target_df["label"] = "1"
            
            non_domestic_target_df = df[~df.gist.fillna("").str.contains("domestic", case=False)]
            non_domestic_target_df = non_domestic_target_df[["gist"]]
            non_domestic_target_df["label"] = "0"
            
            domestic_df = pd.concat([domestic_df, domestic_target_df], ignore_index=True)
            non_domestic_df = pd.concat([non_domestic_df, non_domestic_target_df], ignore_index=True)
    
    return domestic_df, non_domestic_df

def take_random_sample(df, sample_size):
    if len(df) > sample_size:
        return df.sample(sample_size)
    else:
        return df

directory_path = "../../ner/data/output/"

domestic_df, non_domestic_df = process_csv_files(directory_path)

sampled_domestic_df = take_random_sample(domestic_df, 150)
sampled_non_domestic_df = take_random_sample(non_domestic_df, 150)

final_df = pd.concat([sampled_domestic_df, sampled_non_domestic_df], ignore_index=True)

final_df.to_csv("../data/output/labeled_data.csv", index=False)